In [1]:
# http://mnemstudio.org/neural-networks-som4.htm
import math
import random
import sys

EPOCHS = 1000
NUM_CITIES = 5
NUM_NEURONS = NUM_CITIES * 2
NEAR = 0.05
MOMENTUM = 0.995
THETA = 0.5
PHI = 0.5

# Coordinates for example 1.  Ring-shaped map.
CITIES = [[7.0, 2.0], [12.0, 5.0], [9.0, 10.0], [5.0, 10.0], [2.0, 5.0]]

# Coordinates for example 2.
#CITIES = [[3.0, 3.0], [12.0, 2.0], [6.0, 6.0], [7.0, 5.0], [6.0, 12.0]]

class SOM_Class4:
    def __init__(self, numEpochs, numCities, numNeurons, near, momentum, theta, phi, cities):
        self.mNumEpochs = numEpochs
        self.mNumCities = numCities
        self.mNumNeurons = numNeurons
        self.mNear = near
        self.mMomentum = momentum
        self.city = cities
        self.neuronXY = []
        self.weight = []
        self.r = []
        self.gridInterval = 0.0
        self.theta = theta
        self.phi = phi
        return

    def initialize_arrays(self):
        for i in range(self.mNumNeurons):
            self.r.append([0.0] * self.mNumNeurons)

        for i in range(self.mNumNeurons):
            newRow = []
            newRow.append(0.5 + 0.5 * math.cos(self.gridInterval))
            newRow.append(0.5 + 0.5 * math.sin(self.gridInterval))
            self.neuronXY.append(newRow)

            self.gridInterval += math.pi * 2.0 / float(NUM_NEURONS)

            newRow = []
            newRow.append(random.random())
            newRow.append(random.random())
            self.weight.append(newRow)

        self.compute_matrix(self.theta)
        return

    def get_distance(self, index, index2):
        dx = self.neuronXY[index][0] - self.neuronXY[index2][0]
        dy = self.neuronXY[index][1] - self.neuronXY[index2][1]
        return math.sqrt(dx * dx + dy * dy)

    def compute_matrix(self, theta):
        for i in range(self.mNumNeurons):
            self.r[i][i] = 1.0
            for j in range(i + 1, self.mNumNeurons):
                self.r[i][j] = math.exp(-1.0 * (self.get_distance(i, j) * self.get_distance(i, j)) / (2.0 * math.pow(theta, 2)))
                self.r[j][i] = self.r[i][j]
        return

    def find_minimum(self, location1, location2):
        minimumDistance = 100000.0 # or any giant number.
        minimumIndex = -1

        for i in range(self.mNumNeurons):
            distance = (math.pow((location1 - self.weight[i][0]), 2)) + (math.pow((location2 - self.weight[i][1]), 2))
            if distance < minimumDistance:
                minimumDistance = distance
                minimumIndex = i

        return minimumIndex

    def print_ring(self):
        for i in range(self.mNumNeurons):
            sys.stdout.write("(" + "{:03.3f}".format(self.weight[i][0]) + ", " + "{:03.3f}".format(self.weight[i][1]) + ") ")
        sys.stdout.write("\n")
        return

    def algorithm(self):
        index = 0
        minimumIndex = 0
        loc1 = 0.0
        loc2 = 0.0
        count = 0

        while count < self.mNumEpochs:
            # Pick a city for random comparison.
            index = math.floor(random.random() * self.mNumCities)
            loc1 = self.city[index][0] + (random.random() * self.mNear) - self.mNear / 2.0
            loc2 = self.city[index][1] + (random.random() * self.mNear) - self.mNear / 2.0

            minimumIndex = self.find_minimum(loc1, loc2)

            # Update all weights.
            for i in range(self.mNumNeurons):
                self.weight[i][0] += (self.phi * self.r[i][minimumIndex] * (loc1 - self.weight[i][0]))
                self.weight[i][1] += (self.phi * self.r[i][minimumIndex] * (loc2 - self.weight[i][1]))

            self.phi *= self.mMomentum
            self.theta *= self.mMomentum

            self.compute_matrix(self.theta)

            self.print_ring()

            count += 1

        return

if __name__ == '__main__':
    som = SOM_Class4(EPOCHS, NUM_CITIES, NUM_NEURONS, NEAR, MOMENTUM, THETA, PHI, CITIES)
    som.initialize_arrays()
    som.algorithm()


(1.220, 2.944) (0.953, 2.085) (0.565, 1.909) (0.813, 1.209) (0.806, 0.541) (0.758, 0.567) (0.220, 0.836) (0.316, 1.432) (0.514, 1.510) (1.243, 2.211) 
(1.599, 3.975) (1.375, 3.288) (0.916, 2.678) (0.968, 1.714) (0.900, 0.899) (0.839, 0.861) (0.361, 1.171) (0.537, 1.907) (0.878, 2.378) (1.546, 3.361) 
(1.806, 4.488) (1.637, 3.990) (1.185, 3.249) (1.105, 2.143) (0.988, 1.221) (0.914, 1.128) (0.490, 1.471) (0.730, 2.312) (1.157, 3.022) (1.738, 4.034) 
(3.386, 7.214) (3.003, 6.431) (2.110, 4.886) (1.604, 3.150) (1.295, 1.893) (1.171, 1.686) (0.836, 2.124) (1.277, 3.297) (2.089, 4.714) (3.063, 6.457) 
(4.183, 8.575) (3.810, 7.861) (2.803, 6.104) (2.032, 4.009) (1.572, 2.496) (1.406, 2.193) (1.147, 2.710) (1.746, 4.137) (2.787, 5.974) (3.847, 7.876) 
(6.523, 9.282) (5.873, 8.724) (4.260, 7.029) (2.888, 4.749) (2.111, 3.044) (1.857, 2.659) (1.717, 3.243) (2.637, 4.862) (4.247, 6.929) (5.895, 8.733) 
(7.728, 9.623) (7.114, 9.223) (5.366, 7.717) (3.627, 5.382) (2.601, 3.537) (2.272, 3.084) (2.2

(8.988, 9.980) (10.608, 7.233) (11.968, 4.993) (8.947, 3.182) (7.000, 2.008) (4.684, 3.406) (2.017, 5.013) (3.588, 7.639) (5.003, 9.991) (6.506, 9.983) 
(8.988, 9.980) (10.608, 7.233) (11.968, 4.993) (8.933, 3.173) (6.998, 2.007) (4.701, 3.396) (2.017, 5.013) (3.588, 7.639) (5.003, 9.991) (6.506, 9.983) 
(8.988, 9.980) (10.618, 7.218) (11.973, 4.991) (8.954, 3.186) (6.998, 2.007) (4.701, 3.396) (2.017, 5.013) (3.588, 7.639) (5.003, 9.991) (6.506, 9.983) 
(8.990, 9.985) (10.607, 7.237) (11.973, 4.991) (8.954, 3.186) (6.998, 2.007) (4.701, 3.396) (2.017, 5.013) (3.588, 7.639) (5.003, 9.991) (6.523, 9.983) 
(8.990, 9.985) (10.607, 7.237) (11.973, 4.991) (8.941, 3.178) (6.997, 2.008) (4.716, 3.387) (2.017, 5.013) (3.588, 7.639) (5.003, 9.991) (6.523, 9.983) 
(8.990, 9.985) (10.607, 7.237) (11.973, 4.991) (8.941, 3.178) (6.997, 2.008) (4.699, 3.397) (2.014, 5.011) (3.578, 7.623) (5.003, 9.991) (6.523, 9.983) 
(8.990, 9.985) (10.607, 7.237) (11.973, 4.991) (8.930, 3.171) (6.998, 2.008) (4.71

(8.997, 10.003) (10.610, 7.238) (12.004, 5.000) (9.005, 3.215) (6.998, 2.003) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.999, 9.995) (6.570, 9.984) 
(8.997, 10.003) (10.610, 7.238) (12.004, 5.000) (9.005, 3.215) (6.998, 2.003) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.998, 9.995) (6.570, 9.984) 
(8.997, 10.003) (10.610, 7.238) (12.004, 5.000) (9.005, 3.215) (6.998, 2.003) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.999, 9.995) (6.570, 9.984) 
(8.997, 10.003) (10.610, 7.238) (12.004, 5.000) (9.005, 3.215) (6.997, 2.004) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.999, 9.995) (6.570, 9.984) 
(8.998, 10.004) (10.610, 7.238) (12.004, 5.000) (9.005, 3.215) (6.997, 2.004) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.999, 9.995) (6.570, 9.984) 
(8.998, 10.004) (10.610, 7.238) (12.004, 5.002) (9.005, 3.215) (6.997, 2.004) (4.711, 3.390) (2.005, 4.997) (3.578, 7.622) (4.999, 9.995) (6.570, 9.984) 
(8.998, 10.004) (10.610, 7.238) (12.004, 5.002) (9.005, 3.215) (6.997, 2.004

(8.998, 9.998) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.000, 9.998) (6.570, 9.984) 
(8.998, 9.998) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.998, 9.998) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.998, 9.998) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.998, 9.999) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.998, 9.999) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.711, 3.390) (2.004, 4.999) (3.578, 7.622) (5.001, 9.999) (6.570, 9.984) 
(8.998, 9.999) (10.610, 7.238) (12.002, 5.000) (9.005, 3.215) (7.001, 2.002) (4.71

(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 2.000) (4.711, 3.390) (2.004, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 2.000) (4.711, 3.390) (2.004, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.004, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.004, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.004, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.003, 5.000) (3.578, 7.622) (5.001, 9.998) (6.570, 9.984) 
(8.997, 10.000) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999

(8.998, 10.001) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.003, 5.000) (3.578, 7.622) (5.000, 9.999) (6.570, 9.984) 
(8.998, 10.001) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.003, 5.000) (3.578, 7.622) (5.000, 9.999) (6.570, 9.984) 
(8.998, 10.001) (10.610, 7.238) (12.000, 5.001) (9.005, 3.215) (7.001, 1.999) (4.711, 3.390) (2.003, 5.000) (3.578, 7.622) (5.000, 9.999) (6.570, 9.984) 
